# Predictions

This notebook will use several prediction techniques and compare each to the other.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
from utils.prepro import * 

In [3]:
# Constants

_TEST_TRACE_BASE_PATH = "../data/data/parsec_raw_out/canneal/200_50/"
TEST_FAULT_PATH = _TEST_TRACE_BASE_PATH+"fltrace-data-faults-58940-1.out"
TEST_PROCMAP_PATH = _TEST_TRACE_BASE_PATH+"fltrace-data-procmaps-58940.out"
runid = RunIdentifier("canneal",200,50,58940)

# Data loading

In [4]:
df = pd.read_csv(TEST_FAULT_PATH)
df.head()

,tstamp,ip,addr,pages,flags,tid,regs,trace
0,1.728157e+09,7a18c3235bcd,7a1899600000,1,33,58940,134246071009280!14!2097152!8070450532247928832...,0x7a18c321433c|0x7a18c2a42520|0x7a18c3235bcd|0...
1,1.728157e+09,7a18c322f679,7a1899603000,1,33,58940,134246071011712!134246823731956!13424607100947...,0x7a18c321433c|0x7a18c2a42520|0x7a18c322f679|0...
2,1.728157e+09,7a18c2ba0fa0,7a1899604000,1,33,58940,134246071011712!134246823731956!1!0!1!13424607...,0x7a18c321433c|0x7a18c2a42520|0x7a18c2ba0fa0|0...
3,1.728157e+09,7a18c2ba0fa0,7a1899605000,1,33,58940,134246071011712!134246823731956!2!0!0!13424607...,0x7a18c321433c|0x7a18c2a42520|0x7a18c2ba0fa0|0...
4,1.728157e+09,7a18c2ba0fa0,7a1899606000,1,33,58940,134246071011712!134246823731956!3!0!0!13424607...,0x7a18c321433c|0x7a18c2a42520|0x7a18c2ba0fa0|0...


In [ ]:
df,_,epci = preprocess_df(runid,df,Path(TEST_PROCMAP_PATH),get_stats=False,add_missing_ips_to_st=True)

Inspect! canneal_200_50_58940: # of weird ips is 311
